In [ ]:
from datasets import load_dataset, concatenate_datasets
import numpy as np
from pathlib import Path

###### Load NER dataset from Huggingface Hub

In [ ]:
_ds = load_dataset("conll2003")
train, dev, test = _ds["train"], _ds["validation"], _ds["test"]

In [ ]:
train = train.remove_columns(['pos_tags', 'chunk_tags'])
dev = dev.remove_columns(['pos_tags', 'chunk_tags'])
test = test.remove_columns(['pos_tags', 'chunk_tags'])

In [ ]:
#filter sequences with empty tag set
train = train.filter(lambda x: not all([tg==0 for tg in x["ner_tags"]]))
#filter sequences shorter than 8 tokens
train = train.filter(lambda x: len([token for token in x["tokens"]])>=8)
#get subset for test data to reduce model evaluation time 
test = test.select(range(1000))

In [ ]:
labeled, unlabeled = train.train_test_split(train_size=200).values()
unlabeled = unlabeled.remove_columns('ner_tags')

In [ ]:
data_path = './data/NER/conll2003/' 

labeled.to_json(Path(data_path) / 'labeled.json')
unlabeled.to_json(Path(data_path) / 'unlabeled.json')
test.to_json(Path(data_path) / 'test.json')

In [ ]:
import json

with open(Path(data_path) / 'tags.json', 'w') as f:
    json.dump(labeled.features['ner_tags'].feature.names, f)